<a href="https://colab.research.google.com/github/profteachkids/CHE4071_Fall2023/blob/main/TFFeedbackControlLoop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import numpy as np
import jax
jax.config.update('jax_enable_x64',True)
import jax.numpy as jnp
from plotly.subplots import make_subplots
from scipy.optimize import root, minimize
import scipy.signal as sig
from numpy.polynomial import Polynomial as P
from scipy.signal import TransferFunction as TF

In [2]:
def TF2poly(tf):
    return P(tf.num[::-1]), P(tf.den[::-1])

def poly2TF(n,d):
    return TF(n.coef[::-1],d.coef[::-1])

def TFmult(tf1, tf2):
    n1, d1 = TF2poly(tf1)
    n2, d2 = TF2poly(tf2)

    return poly2TF(n1*n2, d1*d2)

def TF_feedback(tf_f, tf_b):
    nf, df = TF2poly(tf_f)
    nb, db = TF2poly(tf_b)

    return poly2TF(nf*db, df*db + nf*nb)


def pid(Kc, taui, taud):
    return TF([Kc*taud*taui, Kc*taui, Kc],[taui,0.])


In [3]:
A1=1.
R1=2.
A2 = 1.
R2=2.
QctoQ2 = TF([1],[A1*R1, 1.])
Q2toH2 = TF([R2],[A2*R2,1.])

In [17]:
rng = np.random.RandomState(123)


In [21]:
t_exp = np.linspace(0,15,50)
TF_proc = TFmult(QctoQ2, Q2toH2)
_, h2_exp = TF_proc.step(T=t_exp)
h2_exp = rng.normal(loc = h2_exp, scale=0.02)

fig = make_subplots()
fig.add_scatter(x=t_exp, y=h2_exp, mode='markers')
fig.update_layout(height=400, width=600, template='plotly_dark')

In [29]:
def FOPTD(v):
    K, tau, theta = v
    y = K*(1-jnp.exp(-(t_exp - theta)/ tau))
    return jnp.where(t_exp<theta, 0., y)

v_guess =(2., 4., 1.)

def SSE(v):
    return jnp.sum((h2_exp - FOPTD(v))**2)

res = minimize(SSE, v_guess)


In [45]:
#We generated synthetic data by using two tank in series transfer function and simulating its response to a step change
#We added random noise to the synthetic data
#Model fitting (nonlinear regression) to obtain parameters for our FOPTD model to obtain K, tau, theta
#ITAE (integrated time-weighted absolute error)

In [30]:
fig = make_subplots()
fig.add_scatter(x=t_exp, y=h2_exp, mode='markers')
fig.add_scatter(x=t_exp, y=FOPTD(res.x), mode='lines')
fig.update_layout(height=400, width=600, template='plotly_dark')

In [37]:
K, tau, theta = res.x
Kc=(0.586*(theta/tau)**(-0.916))/K
taui=tau/(1.03-0.165*(theta/tau))
taud=0.

In [46]:
# Kc=3
# taui=0.5
# taud=0.
TF_fwd = TFmult(pid(Kc,taui, taud),TFmult(QctoQ2, Q2toH2))

/usr/local/lib/python3.10/dist-packages/scipy/signal/_filter_design.py:1746: BadCoefficients:

Badly conditioned filter coefficients (numerator): the results may be meaningless



In [49]:
w = np.logspace(-2,2,100)
w, db, phase= sig.bode(TF_fwd, w)
ar = 10**(db/20)

In [50]:
fig = make_subplots(rows=2, cols=1)
fig.add_scatter(x=w, y=db, row=1, col=1, name='db')
fig.add_scatter(x=w, y=phase, row=2, col=1, name='phase')
fig.update_xaxes(type='log')
fig.update_layout(height=600, width=400, template='plotly_dark')

In [41]:
tsim = np.linspace(0,50,5000)
w=1 #rad/s
e = np.cos(w* tsim)
_, h2, _ = sig.lsim(TF_fwd, e, tsim)
fig=make_subplots()
fig.add_scatter(x=tsim, y=h2, name='output')
fig.add_scatter(x=tsim, y=e, name='input')
fig.update_layout(height=400, width=600, template='plotly_dark')

In [42]:
TF_fb = TF_feedback(TF_fwd, TF([1],[1]))

In [43]:
tplot = np.linspace(0,100, 500)
tplot, h2 = TF_fb.step(T=tplot)

In [44]:
fig=make_subplots()
fig.add_scatter(x=tplot, y=h2)
fig.update_layout(height=400, width=600, template='plotly_dark')